In [11]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
import matplotlib
# matplotlib.use('WebAgg')
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob

### 사용한 모듈들
---
중간의 matplotlib.use('WebAgg')는 프로젝트를 집컴퓨터의 vscode로 시도했기 때문입니다(이미지를 보기 위해 필요했습니다).


In [2]:
def resize_images(img_path):
    images = glob.glob(img_path + "/*.jpg")

    print(len(images), " images to be resized.")
# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size = (28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")

    print(len(images), " images resized.")


def load_data(img_path, number_of_data=1800):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size = 28
    color = 3
    # 이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data * img_size * img_size * color,
                    dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, :, :, :] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 0   # 가위 : 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, :, :, :] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 1   # 바위 : 1
        idx = idx + 1

    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, :, :, :] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 2   # 보 : 2
        idx = idx + 1

    print("학습데이터(x_train)의 이미지 개수는", idx, "입니다.")
    return imgs, labels

### 노드에서 제공해준 함수 입니다.
---
거의 변형 없이 그대로 썼습니다.

In [4]:

# image_dir_path = "./data/paper"
# resize_images(image_dir_path)
# image_dir_path = "./data/rock"
# resize_images(image_dir_path)
# image_dir_path = "./data/scissor"
# resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/"
test_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"

# image_dir_path = "./data"
# test_image_dir_path = "./data/test"

img_number = 5586
test_img_number = 581

(x_train, y_train) = load_data(image_dir_path, img_number)
(x_test, y_test) = load_data(test_image_dir_path, test_img_number)
x_train_norm, x_test_norm = x_train / \
    255.0, x_test / 255.0   # 입력은 0~1 사이의 값으로 정규화

학습데이터(x_train)의 이미지 개수는 5586 입니다.
학습데이터(x_train)의 이미지 개수는 581 입니다.


### 이미지가 로딩되는 부분 입니다.
---
첫 번째 주석부분은 이미지를 리사이즈 할 때 사용했습니다. 당시 test폴더를 구분하지 않아서 한 번에 할 수 있었습니다.

두 번째 주석부분은 집에서 읽을시 사용했던 경로입니다.

받은 이미지를 모두 정리한 결과(못 받는 파일이 있어서 일부 제외했습니다. 구글 퍼미션), 6167개의 사진을 확보했고, 이 중 학습용 이미지를 일정하게 만들고 싶어서 각각 1862개 씩 배정했습니다. 테스트는 각각 보-190, 바위-194, 가위 190개로 배치했습니다.

데이터를 나누는 과정에서 많은 시간이 소요됬고, 새로 배우는 linux/unix 명령어가 있었습니다.

rename 's/^(\d+)/100+$1/e' *.jpg

숫자 이름에 100씩 더하기, 수를 변경해서 덮어쓰기를 막았습니다.

rename -v 's/$/nr/' *

글자 뒤에 첨가하거나 ^를 써서 앞에 첨가했습니다.

In [5]:
n_channel_1 = 32
n_channel_2 = 64
n_dense = 64
n_train_epoch = 3
n_epoch = 25

# print('최소값:', np.min(x_train_norm), ' 최대값:', np.max(x_train_norm))

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(
    n_channel_1, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Conv2D(n_channel_2, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(n_train_epoch, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                102464    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1

### 모델을 설정하는 부분입니다.
---
가위바위보가 숫자보다 조금 더 많은 특징을 파악해야한다고 생각해서 전반적인 값을 2배정도 상향했습니다.  
그리고 가위, 바위 보 이므로 3개의 카테고리이고,
마지막 n_epoch은 학습 횟수인데, 20번까지 정확도가 가감되면서 증가하다가,  
23정도에서 1.000에 이르렀으므로, 25에서 멈추기로 했습니다.  

input_shape의 마지막 매개변수를 3으로 변경했습니다. 그림이 흑백이 아니라서 3이라고 생각합니다.  

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=n_epoch)

Epoch 1/25
175/175 [==============================] - 7s 18ms/step - loss: 1.0551 - accuracy: 0.4260
Epoch 2/25
175/175 [==============================] - 1s 3ms/step - loss: 0.6521 - accuracy: 0.7414
Epoch 3/25
175/175 [==============================] - 1s 4ms/step - loss: 0.3347 - accuracy: 0.8821
Epoch 4/25
175/175 [==============================] - 1s 3ms/step - loss: 0.2206 - accuracy: 0.9291
Epoch 5/25
175/175 [==============================] - 1s 3ms/step - loss: 0.1436 - accuracy: 0.9575
Epoch 6/25
175/175 [==============================] - 1s 3ms/step - loss: 0.0886 - accuracy: 0.9774
Epoch 7/25
175/175 [==============================] - 1s 4ms/step - loss: 0.0810 - accuracy: 0.9772
Epoch 8/25
175/175 [==============================] - 1s 4ms/step - loss: 0.0429 - accuracy: 0.9919
Epoch 9/25
175/175 [==============================] - 1s 3ms/step - loss: 0.0377 - accuracy: 0.9922
Epoch 10/25
175/175 [==============================] - 1s 3ms/step - loss: 0.0220 - accuracy: 0.995

### 모델을 생성하는 부분입니다.
---
노드에서는 x_train_norm의 모양을 변경해줘야 했지만, 현 프로젝트에서는 그럴 필요가 없었습니다.

In [8]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

19/19 - 2s - loss: 0.0300 - accuracy: 0.9845
test_loss: 0.030021443963050842 
test_accuracy: 0.9845094680786133


### 실제 테스트 부분입니다.
---

학습시의 1.0의 정확도를 보였지만 테스트에서는 1.0의 정확도를 이루지는 못 했습니다.

In [12]:
# 오류확인용
predicted_result = model.predict(x_test_norm)  # model이 추론한 확률값.
predicted_labels = np.argmax(predicted_result, axis=1)

import random
wrong_predict_list = []
for i, _ in enumerate(predicted_labels):
    # i번째 test_labels과 y_test이 다른 경우만 모아 봅시다.
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)

# wrong_predict_list 에서 랜덤하게 5개만 뽑아봅시다.
samples = random.choices(population=wrong_predict_list, k=5)

for n in samples:
    print("예측확률분포: " + str(predicted_result[n]))
    print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
    plt.imshow(x_test[n], cmap=plt.cm.binary)
    plt.show()

예측확률분포: [1.3894066e-01 3.3731008e-05 8.6102557e-01]
라벨: 0, 예측결과: 2
To view figure, visit http://127.0.0.1:8988
예측확률분포: [9.7298878e-01 1.6710766e-09 2.7011245e-02]
라벨: 2, 예측결과: 0
To view figure, visit http://127.0.0.1:8988
예측확률분포: [8.2305622e-01 1.7694248e-01 1.3111560e-06]
라벨: 1, 예측결과: 0
To view figure, visit http://127.0.0.1:8988
예측확률분포: [0.4974044  0.46261185 0.03998368]
라벨: 1, 예측결과: 0
To view figure, visit http://127.0.0.1:8988
예측확률분포: [9.7298878e-01 1.6710766e-09 2.7011245e-02]
라벨: 2, 예측결과: 0
To view figure, visit http://127.0.0.1:8988


   ### 잘 못 검증된 사진 확인 부분입니다
   ---
   육안으로 확인하기 위해 시도했습니다.  
   노드의 코드에서 변경된 것은 없습니다.  
   육안으로 식별이 잘 안 되는 사진이 많지만, 크기를 줄여서 그런 것 같습니다.  
   컴퓨터도 해상도 때문에 이미지크기에 정확도가 영향 받을 수 있다는 생각을 했습니다.  

## 마치며
---
처음으로 인공지능 프로그램을 만들어 볼 수 있어서 좋았습니다.  
인공지능 프로그램이 기본적으로 갖추는 구조를 배울 수 있었던 것 같습니다.  
모델에 대해서 이해를 잘 하지 못 한 것 같지만, 추후 보충해 나갈 수 있을 것이라고 기대합니다.  
